## **Politcal Donations by Professional Sport Team Owners Analysis**

This dataset contains all politcal donations by American sport team owners between the 2016-2020 election cycles. The dataset was found on Kaggle and was uploaded by Rahul Kapoor, data was originally sourced from the Federal Elections Commission and OpenSecrets. 

You can find the data set here: [https://www.kaggle.com/datasets/rahul253801/political-donations-by-american-sports-owners](https://) 

**Notes on Dataset:**

>* Owners are from NFL, NBA, WNBA, MLB, NHL, and NASCAR
>* Only politcal donations while owner was with team are recorded 
>* Fields in Table: Owner, Team, League, Donation Recipient, Amount, Year, and Party

Hypothesis:
>* Out of the professional sports leagues my guess is that NFL owners contributed the most to candidates between 2016-2020
>* Between the two political parties, my gut tells me that more donations are going to lean Republican but I may be surprised
>* I want to analyze the top contributing team owners regardless of league and see if there are any similarites. My thinking is that you'll see owners/teams that have a global brand and are in big metropolitian markets (ex. Los Angeles Lakers or Dallas Cowboys)
>* In terms of type of donation; were there more local, state, or federal contributions? I think we may see more state contributions on average then federal since state law would have more of an effect on their team's operation then federal law
>* I know there are certain contribution limits so I'm curious if we will see constant amounts numerous times, my guess is that we will
>*Since the dataset has two federal election included I want to compare 2016 to 2020. I think we'll see an increase in total contributions and the share of contributions between the two political parties may have changed

In [2]:
import pandas as pd 
import numpy as np
import re


# using pandas to create dataframe on .csv to perform some analysis
owner_contributions = pd.read_csv('sports-political-donations.csv', sep = ',')
owner_contributions.info()
owner_contributions.keys()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2798 entries, 0 to 2797
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Owner          2798 non-null   object
 1   Team           2798 non-null   object
 2   League         2798 non-null   object
 3   Recipient      2798 non-null   object
 4   Amount         2798 non-null   object
 5   Election Year  2798 non-null   int64 
 6   Party          2789 non-null   object
dtypes: int64(1), object(6)
memory usage: 153.1+ KB


Index(['Owner', 'Team', 'League', 'Recipient', 'Amount', 'Election Year',
       'Party'],
      dtype='object')

**QUERY 1:**

**Looking at total campaign contributions by owners grouped by the league their team plays in**

In [3]:
# So looks like Amount is in object data type instead of a numeric data type, need to change that first

#owner_contributions['Amount'] = pd.to_numeric(owner_contributions['Amount'])

# ^ error message for line above because there's special characters in Amount column( $ and ,). Need to get rid of those
# first somehow

owner_contributions['Amount'] = owner_contributions['Amount'].str.replace(r'[($)|(,)]', '')
owner_contributions['Amount'] = owner_contributions['Amount'].astype(int)
#owner_contributions.info()

# Woohoo we got a numeric data type now, lets run this original line again 

league_donations = owner_contributions.groupby('League').sum()[['Amount']].sort_values('Amount', ascending = False)
league_donations

#Looks like some weird league groupings, must be an owner that owns multiple teams in different leagues?



<ipython-input-3-9bd684428063>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  owner_contributions['Amount'] = owner_contributions['Amount'].str.replace(r'[($)|(,)]', '')


,Amount
League,
MLB,19485673
NBA,6501779
NHL,6425482
NFL,5926820
"NBA, NHL",2166320
WNBA,1883942
"NBA, MLB",1824275
"NBA, WNBA",800100
NASCAR,728393


In [ ]:
# I want to look at entries that have multiple leagues listed 
multi_team = owner_contributions[owner_contributions['League'] == 'NBA, NHL']
multi_owners = multi_team.groupby('Owner').sum()[['Amount']]
multi_team 

# I choose the first two league variation I saw, and it does in fact look like it is from owner's having two different teams,
# or even three in some cases. This makes it tricky for me to determine which league to attribute the contributions
# too. As it turns out this isn't a very good query because of this, way too many contributions were made by
# multi team owners to get accurate analysis. I think it is safe too say that MLB owners donated far and away the most though
# with roughly $13 million more in campaign contributions than the second closest league 

**QUERY 2:**

**Looking at whether donations skew Republican or Democrat among team owners**

In [5]:

party_skew = owner_contributions.groupby('Party').sum()[['Amount']].sort_values('Amount', ascending = False)
#party_skew

# That suprises me that Republican party recieved so many more campaign contributions, kind of want to see what that
# looks like broken down by the two federal elections and the 2018 midterm cycle

contributions_2016 = owner_contributions[owner_contributions['Election Year'] == 2016]
party_skew_2016 = contributions_2016.groupby('Party').sum()[['Amount']].sort_values('Amount', ascending = False)
party_skew_2016

contributions_2018 = owner_contributions[owner_contributions['Election Year'] == 2018]
party_skew_2018 = contributions_2018.groupby('Party').sum()[['Amount']].sort_values('Amount', ascending = False)
party_skew_2018

contributions_2020 = owner_contributions[owner_contributions['Election Year'] == 2020]
party_skew_2020 = contributions_2020.groupby('Party').sum()[['Amount']].sort_values('Amount', ascending = False)
party_skew_2018

party_skew['2016'] = party_skew_2016['Amount']
party_skew['2018'] = party_skew_2018['Amount']
party_skew['2020'] = party_skew_2020['Amount']

party_skew # now has total broken down  for each major election cycle 2016, 2018, 2020


# Pretty constant numbers in sense of share of contributions. $13 million, $11 million, and $10 million to Republican candidates
# and $4 million, $4 million, and $ 2 million for Democratic candidates

# Suprises me how little Democratic contributions there were relative to Republican over 3 major election cycles

#party_skew.to_csv('party_share_by_year.csv')

,Amount,2016,2018,2020
Party,,,,
Republican,34246016,12940514,11282570,10022932.0
Democrat,10113639,4065094,4174212,1874333.0
Bipartisan,1474699,578050,540083,356566.0
"Bipartisan, but mostly Republican",271093,83638,99103,88352.0
Independent,13500,2700,10800,NaN
"Bipartisan, but mostly Democratic",10800,2000,6000,2800.0


### **QUERY 3:** 
**Examining which team owners contributed  the most to different campaigns** 

In [ ]:
top_donors = owner_contributions.groupby(['Owner', 'Team', 'League']).sum()[['Amount']].sort_values('Amount', ascending = False)
top_donors['% of Total'] = top_donors['Amount']/ sum(top_donors['Amount'])
top_donors.head(15)

# So this is actually pretty interesting, Charles Johnson could be codename the Giver, he was responsible for about
# a quarter of total campaign contributions, donating a whopping $11 million, the next closest owner was Dan DeVos who 
# donated $2.3 million and was more in the normal range of 5% or below total contribution share


,,,Amount,% of Total
Owner,Team,League,,
Charles Johnson,San Francisco Giants,MLB,11035700,0.234909
Dan DeVos,Orlando Magic,NBA,2310400,0.049180
Peter Angelos,Baltimore Orioles,MLB,2092772,0.044547
Jerry Reinsdorf,"Chicago Bulls, Chicago White Sox","NBA, MLB",1797300,0.038258
Philip F. Anschutz,Los Angeles Kings,NHL,1776700,0.037819
Jimmy and Susan Haslam,Cleveland Browns,NFL,1737200,0.036978
Laura Ricketts,Chicago Cubs,MLB,1718124,0.036572
James L. Dolan,"New York Knicks, New York Rangers","NBA, NHL",1610000,0.034271
Dan Gilbert,Cleveland Cavaliers,NBA,1510200,0.032146


**QUERY 4:** 

**Which type of contribution do you see the most? State, Local, Federal?**

In [ ]:
# This query opened my eyes to how difficult it is track contributions on this scale, at least in data analyst lense.
# There are so many (my guess intentional) naming varitations that give no indication as to what the campaign/cause/committee is 
# by their name alone, that it becomes very tedious to try and group into meaningful groups. And regardless of that it seems
# fair size of the contributions were to PAC's and not individual campaigns. Contribtuions to PAC's are hard to classify into local, state,
# or federal ends because their funds are used for a myriad of different purposes. Going to go in different direction
# and look at top PACS and top recipients overall, in count and amount 

top_recipients = owner_contributions.groupby('Recipient').size().sort_values(ascending = False)
recipient_amount = owner_contributions.groupby('Recipient').sum()[['Amount']]

#First getting PAC donation counts
PAC_count = owner_contributions[(owner_contributions['Recipient'].str.contains('PAC')) | 
                               (owner_contributions['Recipient'].str.contains('Political Action', case = False))].groupby('Recipient').size()

PAC_breakdown = PAC_count.to_frame()

#Now seeing PACs sorted by donation $ amount (high to low)
PAC_amount = owner_contributions[(owner_contributions['Recipient'].str.contains('PAC')) | 
                                (owner_contributions['Recipient'].str.contains('Political Action', 
                                case = False))].groupby('Recipient').sum()[['Amount']].sort_values('Amount', ascending = False)
PAC_amount

#now adding amount figure to previous data frame as a newcolumn to have info side by side
# renaming columns to clean it up a bit then sorting by $ amount
PAC_breakdown['Total Amount ($)'] = PAC_amount['Amount']
PAC_breakdown.rename(columns = {0: 'Donation Count'}, inplace = True)
PAC_by_amount = PAC_breakdown.sort_values('Total Amount ($)', ascending = False)

# now sorting by donation count in new dataframe stored in separate variable
PAC_by_count = PAC_count.sort_values(ascending = False).to_frame()
PAC_by_count.rename(columns = {0: 'Donation Count'}, inplace = True)
PAC_by_count['Total Amount ($)'] = PAC_amount['Amount']

#now have two dataframes one sorted by PAC donation count and the other by PAC $ amount
PAC_by_count.head(15)
PAC_by_amount.head(15)

# finally going to insert amount back into top recipients and see what that looks like
top_recipients = top_recipients.to_frame()
top_recipients['Total Amount ($)'] = recipient_amount['Amount']
top_recipients.rename(columns = {0:'Donation Count'}, inplace = True)
recipient_by_count = top_recipients
recipient_by_amount = top_recipients.sort_values('Total Amount ($)', ascending = False)

#now have two dataframes one sorted by recipient donation count and the other by their $ amount
recipient_by_count
recipient_by_amount



**QUERY 5:**

**Examining whether you see constant figure which coincides with the maximum 
contribution limit set by FEC**



In [8]:
most_common_donations = owner_contributions.groupby('Amount').size().sort_values(ascending = False)
most_common_donations.head(15)

# $2700 coincides with the limit set by FEC that an individual can give to a candidate
# That number rose to $2800 in 2019-2020  

# $5000  and $10000 reflect the limits for contributions to PACs and political party committees (state/district/local)

# Any donations larger than that should fall under caps for national party committee contributions but that's where we see
# the $100,000+ contributions

# looking at largest donation sizes now
large_amounts = owner_contributions.sort_values('Amount', ascending = False)
top_ten_dontations = large_amounts.head(10)
top_ten_dontations
#top_ten_dontations.to_csv('top_single_contributions.csv')


most_common_donations


Amount
2700       437
5400       384
5000       264
1000       245
5600       226
          ... 
8500         1
8682         1
9000         1
9216         1
1850000      1
Length: 244, dtype: int64

**QUERY 6:**

**Examining whether there was an increase or decrease in contributions between 2016 and 2020**

In [ ]:
donation_growth = owner_contributions[(owner_contributions['Election Year'] == 2016) | 
                  (owner_contributions['Election Year'] == 2020) | (owner_contributions['Election Year'] == 2018)].groupby('Election Year').sum()[['Amount']]
donation_growth

# I'm suprised that the $ amount of donations dropped by $5 million
# I'm going to add 2018 in now just to see that as well

# hmm interesting you saw more donations in 2018 (a midterm) than 2020 (national election)

,Amount
Election Year,
2016,17679696
2018,16932768
2020,12366233
